In [1]:
from toy_cnn_for_preprocessed import make_model
from retrieve_data import train_data, validation_data, test_data

In [2]:
Xg_train, Xl_train, label_train = train_data()
Xg_valid, Xl_valid, label_valid = validation_data()
Xg_test, Xl_test, label_test = test_data()

2025-10-04 14:41:10.426708: I tensorflow/core/kernels/data/tf_record_dataset_op.cc:390] TFRecordDataset `buffer_size` is unspecified, default to 262144
2025-10-04 14:41:10.533336: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2025-10-04 14:41:10.648470: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2025-10-04 14:41:10.873828: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2025-10-04 14:41:11.324348: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [3]:
import numpy as np

# Add channel dimension for Conv1D
Xg_train = np.expand_dims(Xg_train, axis=-1)  # shape becomes (num_samples, 2001, 1)
Xl_train = np.expand_dims(Xl_train, axis=-1)  # shape becomes (num_samples, 201, 1)

Xg_valid = np.expand_dims(Xg_valid, axis=-1)
Xl_valid = np.expand_dims(Xl_valid, axis=-1)

Xg_test = np.expand_dims(Xg_test, axis=-1)
Xl_test = np.expand_dims(Xl_test, axis=-1)


In [4]:
Xg_train.shape

(12589, 2001, 1)

In [5]:
print(np.count_nonzero(label_train), label_train.shape)
print(np.count_nonzero(label_valid), label_valid.shape)
print(np.count_nonzero(label_test), label_test.shape)

2885 (12589,)
355 (1574,)
360 (1574,)


In [6]:
np.argwhere(label_train == 1)

array([[    0],
       [    3],
       [    4],
       ...,
       [12586],
       [12587],
       [12588]], shape=(2885, 1))

In [7]:
model = make_model()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ global_view         │ (None, 2001, 1)   │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ local_view          │ (None, 201, 1)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d (Conv1D)     │ (None, 1997, 4)   │         24 │ global_view[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_1 (Conv1D)   │ (None, 199, 2)    │          8 │ local_view[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d       │ (None, 998, 4)    │          0 │ conv1d[0][0]      │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d_1     │ (None, 99, 2)     │          0 │ conv1d_1[0][0]    │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 3992)      │          0 │ max_pooling1d[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_1 (Flatten) │ (None, 198)       │          0 │ max_pooling1d_1[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 64)        │    255,552 │ flatten[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 32)        │      6,368 │ flatten_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 96)        │          0 │ dense[0][0],      │
│ (Concatenate)       │                   │            │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 64)        │      6,208 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 1)         │         65 │ dense_2[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 268,225 (1.02 MB)

 Trainable params: 268,225 (1.02 MB)

 Non-trainable params: 0 (0.00 B)

In [8]:
from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history = model.fit(
    [Xg_train, Xl_train], label_train,
    epochs=20,
    batch_size=32,
    validation_data=([Xg_valid, Xl_valid], label_valid),
    callbacks=[early_stop]
)


Epoch 1/20
394/394 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8990 - loss: 0.2320 - val_accuracy: 0.9187 - val_loss: 0.1971
Epoch 2/20
394/394 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9371 - loss: 0.1534 - val_accuracy: 0.9352 - val_loss: 0.1688
Epoch 3/20
394/394 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9474 - loss: 0.1328 - val_accuracy: 0.9257 - val_loss: 0.1864
Epoch 4/20
394/394 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9553 - loss: 0.1137 - val_accuracy: 0.9352 - val_loss: 0.1680
Epoch 5/20
394/394 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9621 - loss: 0.0962 - val_accuracy: 0.9263 - val_loss: 0.2034
Epoch 6/20
394/394 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9689 - loss: 0.0818 - val_accuracy: 0.9339 - val_loss: 0.2082
Epoch 7/20
394/394 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9766 - loss: 0.0613 - val_accuracy: 0.9238 - val_loss: 0.2728
Epoch 8/20
394/394 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9782 - loss: 0.0564 - val_accuracy: 0.

In [9]:
test_loss, test_acc = model.evaluate([Xg_test, Xl_test], label_test)
print("Test Accuracy:", test_acc)

50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9441 - loss: 0.1388 
Test Accuracy: 0.9440914988517761
